In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.datasets import cifar10, imdb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import scipy
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import json
from datetime import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

def preprocess_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    # Select classes 3 and 5
    mask_train = np.logical_or(y_train == 3, y_train == 5).ravel()
    X_train = X_train[mask_train]
    y_train = y_train[mask_train]

    mask_test = np.logical_or(y_test == 3, y_test == 5).ravel()
    X_test = X_test[mask_test]
    y_test = y_test[mask_test]

    # Limit to first 1000 training samples
    X_train = X_train[:100]
    y_train = y_train[:100]
    X_test = X_test[:10]
    y_test = y_test[:10]
    
    # Change labels to binary
    y_train = np.where(y_train == 3, 0, 1)
    y_test = np.where(y_test == 3, 0, 1)

    # Convert to grayscale
    X_train = tf.image.rgb_to_grayscale(X_train).numpy()
    X_test = tf.image.rgb_to_grayscale(X_test).numpy()

    # Normalize by dividing by 255
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Further normalize by subtracting mean and dividing by standard deviation
    mean_image = np.mean(X_train, axis=0)
    std_image = np.std(X_train, axis=0)
    X_train = (X_train - mean_image) / std_image
    X_test = (X_test - mean_image) / std_image

    return X_train, y_train, X_test, y_test

def preprocess_imdb():
    (X_train, y_train), (X_test, y_test) = imdb.load_data(path="imdb.npz",num_words=None,skip_top=0, maxlen=None,seed=1,start_char=1,oov_char=2,index_from=3)
    # Odwróć indeks słownika IMDB, aby uzyskać mapowanie z indeksu na słowo
    word_index = imdb.get_word_index()
    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

    # Dekoduj recenzje do tekstów
    decoded_train = [" ".join([reverse_word_index.get(i - 3, "?") for i in review]) for review in X_train] #Dlaczego i - 3? - trzy pierwsze indeksy są zarezerwowane dla wartości specjalnych: start sekwencji, nieznane słowo i koniec sekwencji, więc indeksy rzeczywistych słów w słowniku są przesunięte o 3.
    decoded_test = [" ".join([reverse_word_index.get(i - 3, "?") for i in review]) for review in X_test]

    vectorizer = TfidfVectorizer(max_features=1000)
    X_train = vectorizer.fit_transform(decoded_train)
    X_test = vectorizer.transform(decoded_test)
    
    # Limit to first 1000 training samples
    X_train = X_train[:100]
    y_train = y_train[:100]
    X_test = X_test[:10]
    y_test = y_test[:10]

    return X_train, y_train, X_test, y_test

def preprocess_breast_cancer():
    data = load_breast_cancer()
    X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)
    
    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Fit on training set only and transform both train and test sets
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, y_train, X_test, y_test


def split_labeled_unlabeled(X, y, labeled_size):
    X_labeled, X_unlabeled, y_labeled, _ = train_test_split(X, y, train_size=labeled_size, random_state=42, stratify=y)
    return X_labeled, y_labeled, X_unlabeled

def evaluate_model(model, X_test, y_test):
    print(f"Testing model")
    y_pred = model.predict(X_test)
    if hasattr(model.predict, '__code__') and 'Transductive' in model.predict.__code__.co_varnames:
        y_pred = model.predict(X=X_test, Transductive=False)
    else:
        y_pred = model.predict(X_test)
    print(y_pred)
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred)
    }

def train_and_evaluate(dataset_name, model, X_train, y_train, X_unlabeled, X_test, y_test):
    print(f"Training model")
          
    if dataset_name == 'cifar10':
        y_train = y_train.ravel()
        y_test = y_test.ravel()
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_unlabeled = X_unlabeled.reshape(X_unlabeled.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)

        # Konwersja rzadkich macierzy do gęstych tablic dla danych IMDB
    elif dataset_name == 'imdb':
        X_train = X_train.toarray() if scipy.sparse.issparse(X_train) else X_train
        X_unlabeled = X_unlabeled.toarray() if scipy.sparse.issparse(X_unlabeled) else X_unlabeled
        X_test = X_test.toarray() if scipy.sparse.issparse(X_test) else X_test

    print(f"X_train shape: {X_train.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_train shape: {y_train.shape}")
    start_time = time.time()
    model.fit(X=X_train, y=y_train, unlabeled_X=X_unlabeled)
    training_time = time.time() - start_time
    results = evaluate_model(model, X_test, y_test)
    results['time'] = training_time
    return results

def save_results(results, model_name, dataset_name, model_params, labeled_size):
    filename = f"experiments_results/{model_name}_{dataset_name}.json"
    data_to_save = {
        "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "algorithm": model_name,
        'dataset': dataset_name,
        'labeled_size': labeled_size,
        "hyperparameters": str(model_params),
        "results": results
    }
    with open(filename, 'a') as f, open('experiments_results/all_results.json', 'a') as g:
        json.dump(data_to_save, f, indent=4)
        json.dump(data_to_save, g, indent=4)

def knn_factory():
    return KNeighborsClassifier(n_neighbors=7)

def random_forest_factory():
    return RandomForestClassifier()

def linear_svc_factory():
    return LinearSVC(max_iter=10**4)


In [2]:
from LAMDA_SSL.Algorithm.Classification.Tri_Training import Tri_Training
from LAMDA_SSL.Algorithm.Classification.Assemble import Assemble
from LAMDA_SSL.Algorithm.Classification.SemiBoost import SemiBoost
from LAMDA_SSL.Algorithm.Classification.LapSVM import LapSVM
from LAMDA_SSL.Algorithm.Classification.VAT import VAT
from LAMDA_SSL.Algorithm.Classification.TSVM import TSVM
from LAMDA_SSL.Algorithm.Classification.LabelPropagation import LabelPropagation
from LAMDA_SSL.Algorithm.Classification.TemporalEnsembling import TemporalEnsembling
from LAMDA_SSL.Algorithm.Classification.MeanTeacher import MeanTeacher


# Inicjalizacja modeli i danych
datasets = {
            'cifar10' : preprocess_cifar10, 
            'imdb' : preprocess_imdb, 
            'breast_cancer': preprocess_breast_cancer
            }

models = {
          'Tri_Training' : Tri_Training,
          'Assemble' : Assemble,
          'SemiBoost' : SemiBoost,
          'LapSVM' : LapSVM,
#           'TemporalEnsembling' : TemporalEnsembling, 
#           'MeanTeacher' : MeanTeacher,
#           'VAT' : VAT,
          'TSVM' : TSVM,
          'LabelPropagation' : LabelPropagation
          }

models_hyperparameters = {
              'Tri_Training' : {
                  'base_estimator': knn_factory,
                  'base_estimator_2': random_forest_factory,
                  'base_estimator_3': linear_svc_factory
                                }
              ,'Assemble' : {
                  'T' : 10,
                  'base_estimator' : random_forest_factory
              }
              ,'SemiBoost' : {
                  'T' : 10,
                  'base_estimator' : random_forest_factory
              }
              ,'LapSVM' : {                  
              }
              ,'VAT' : {
              }
              ,'TSVM' : {
              }
              ,'LabelPropagation' : {
              }
              ,'TemporalEnsembling' : {
              }
              ,'MeanTeacher' : {
                  
              }
            }
results = {
      'Tri_Training' : {},
      'Assemble' : {},
      'SemiBoost' : {},
      'LapSVM' : {},
      'VAT' : {},
      'TSVM' : {},
      'LabelPropagation' : {},
      'TemporalEnsembling' : {},
      'MeanTeacher' : {}
}

labeled_size = 0.3

def train_model(model_name, model_function, dataset_name, preprocess_function, labeled_size,
                models_hyperparameters=models_hyperparameters, results=results):
    print(f"Alghoritm: {model_name}")
    print(f"Dataset: {dataset_name}")
    X_train, y_train, X_test, y_test = preprocess_function()
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")

    model_params = models_hyperparameters.get(model_name, {})
    model_params = {k: v() if callable(v) else v for k, v in model_params.items()}
    model = model_function(**model_params)

    results[model_name][dataset_name] = train_and_evaluate(dataset_name, model, X_labeled, y_labeled, X_unlabeled, X_test, y_test)    
    print(f"{dataset_name} Results:", results[model_name][dataset_name])
    print()
    save_results(results[model_name][dataset_name], model_name, dataset_name, model_params, labeled_size)

In [3]:
for model_name, model_function in models.items():
    for dataset_name, preprocess_function in datasets.items():
        train_model(model_name, model_function, dataset_name, preprocess_function, labeled_size)


Alghoritm: Tri_Training
Dataset: cifar10
X_labeled shape: (30, 32, 32, 1), X_unlabeled shape: (70, 32, 32, 1)
y_labeled shape: (30, 1)
Training model
X_train shape: (30, 1024), X_unlabeled shape: (70, 1024)
y_train shape: (30,)
Testing model
[0 1 0 0 1 0 1 0 0 1]
cifar10 Results: {'accuracy': 0.3, 'precision': 0.5, 'recall': 0.2857142857142857, 'f1_score': 0.36363636363636365, 'time': 0.8894035816192627}

Alghoritm: Tri_Training
Dataset: imdb
X_labeled shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_labeled shape: (30,)
Training model
X_train shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_train shape: (30,)
Testing model
[0 0 0 0 0 0 0 0 0 0]
imdb Results: {'accuracy': 0.9, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'time': 0.617443323135376}

Alghoritm: Tri_Training
Dataset: breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Testing model
[1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0
 1 1 0]
breast_cancer Results: {'accuracy': 0.956140350877193, 'precision': 0.9583333333333334, 'recall': 0.971830985915493, 'f1_score': 0.965034965034965, 'time': 0.5867249965667725}

Alghoritm: Assemble
Dataset: cifar10
X_labeled shape: (30, 32, 32, 1), X_unlabeled shape: (70, 32, 32, 1)
y_labeled shape: (30, 1)
Training model
X_train shape: (30, 1024), X_unlabeled shape: (70, 1024)
y_train shape: (30,)
Testing model
[0 1 0 0 1 0 0 0 0 1]
cifar10 Results: {'accuracy': 0.2, 'precision': 0.3333333333333333, 'recall': 0.14285714285714285, 'f1_score': 0.2, 'time': 3.4909586906433105}

Alghoritm: Assemble
Dataset: imdb
X_labeled shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_labeled shape: (30,)
Training model
X_train shape: (30, 1000), X_unlabeled s

c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


X_labeled shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_labeled shape: (30,)
Training model
X_train shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_train shape: (30,)
Testing model
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
imdb Results: {'accuracy': 0.9, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'time': 0.0079803466796875}

Alghoritm: LapSVM
Dataset: breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)


...

Testing model
[1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.]
breast_cancer Results: {'accuracy': 0.9473684210526315, 'precision': 0.9452054794520548, 'recall': 0.971830985915493, 'f1_score': 0.9583333333333334, 'time': 0.32013988494873047}

Alghoritm: TSVM
Dataset: cifar10
X_labeled shape: (30, 32, 32, 1), X_unlabeled shape: (70, 32, 32, 1)
y_labeled shape: (30, 1)
Training model
X_train shape: (30, 1024), X_unlabeled shape: (70, 1024)
y_train shape: (30,)
Testing model
[0 1 0 0 1 0 1 0 0 1]
cifar10 Results: {'accuracy': 0.3, 'precision': 0.5, 'recall': 0.2857142857142857, 'f1_score': 0.36363636363636365, 'time': 0.4961833953857422}

Alghoritm: TSVM
Dataset: imdb
X_labeled shape: (30, 1000)

c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer


X_labeled shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_labeled shape: (30,)
Training model
X_train shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_train shape: (30,)
Testing model
[0 0 0 0 0 0 0 0 0 0]
imdb Results: {'accuracy': 0.9, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'time': 0.0029921531677246094}

Alghoritm: LabelPropagation
Dataset: breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Testing model
[1 0 0 1 1 0 0 0 0 1 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 1 0 1 0
 1 1 1 1 1 1 0 1 0 0 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0
 1 1 0]
breast_cancer Results: {'accuracy': 0.9210526315789473, 'precision': 0.918918918918919, 'recall': 0.9577464788732394, 'f1_score': 0.9379310344827587, 'time': 0.46180105209350586}



In [56]:
for dataset_name, preprocess_function in datasets.items():
    train_model('TSVM', TSVM, dataset_name, preprocess_function, labeled_size)

Alghoritm: TSVM
Dataset: cifar10
X_labeled shape: (30, 32, 32, 1), X_unlabeled shape: (70, 32, 32, 1)
y_labeled shape: (30, 1)
Training model
X_train shape: (30, 1024), X_unlabeled shape: (70, 1024)
y_train shape: (30,)
Testing model
[0 1 0 0 1 0 1 0 0 1]
cifar10 Results: {'accuracy': 0.3, 'precision': 0.5, 'recall': 0.2857142857142857, 'f1_score': 0.36363636363636365, 'time': 0.4727356433868408}

Alghoritm: TSVM
Dataset: imdb
X_labeled shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_labeled shape: (30,)
Training model
X_train shape: (30, 1000), X_unlabeled shape: (70, 1000)
y_train shape: (30,)
Testing model
[0 0 0 0 0 0 1 0 0 0]
imdb Results: {'accuracy': 0.8, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'time': 0.4807136058807373}

Alghoritm: TSVM
Dataset: breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
Testing model
[1 0 0 1 1 0 0 0 0 1 

#  <font color='green'>ASSEMBLE</font>

### Hiperparametry

1. **base_estimator** - Podstawowy klasyfikator używany w uczeniu zespołowym. Dowolny klasyfikator zgodny z interfejsem sklearn (np. DecisionTreeClassifier, KNeighborsClassifier).


2. **T** - Liczba podstawowych klasyfikatorów, równocześnie jest to liczba iteracji.
    Możliwe wartości: Liczby całkowite większe od 0.


3. **alpha** - Waga każdej próbki przy aktualizacji rozkładu próbkowania.
    Możliwe wartości: Liczby rzeczywiste większe od 0.


4. **beta** - Używany do inicjalizacji rozkładu próbkowania danych oznaczonych i nieoznaczonych.
    Możliwe wartości: Liczby rzeczywiste w przedziale [0, 1].


In [8]:
from LAMDA_SSL.Algorithm.Classification.Assemble import Assemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
#datasets = ['cifar10','imdb', 'breast_cancer']
datasets = ['imdb', 'breast_cancer']
labeled_size = 0.3
base_estimators = [KNeighborsClassifier(n_neighbors=7), RandomForestClassifier(), LinearSVC(max_iter=10**4)]
T=10
models = {dataset: Assemble(T=T,base_estimator=base_estimators[1]) for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing imdb
X_labeled shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_labeled shape: (7500,)
Training model on imdb
X_train shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_train shape: (7500,)
imdb Results: {'accuracy': 0.8132, 'precision': 0.8071069971760276, 'recall': 0.82312, 'f1_score': 0.8150348542458808}
Training Time imdb: 460.310693025589

Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
breast_cancer Results: {'accuracy': 0.956140350877193, 'precision': 0.9342105263157895, 'recall': 1.0, 'f1_score': 0.9659863945578232}
Training Time breast_cancer: 3.2257578372955322



#  <font color='green'>Semiboost</font>

### Hiperparametry 

1. **base_estimator**
   - Opis: Bazowy nadzorowany uczeń używany w algorytmie.
   - Możliwe wartości: Dowolny klasyfikator zgodny z interfejsem scikit-learn.


2. **n_neighbors**
   - Opis: Liczba sąsiadów w funkcji kernela 'knn'.
   - Możliwe wartości: Dowolna liczba całkowita większa od 0.


3. **n_jobs**
   - Opis: Liczba równoległych zadań w funkcji kernela 'knn'.
   - Możliwe wartości: Dowolna liczba całkowita lub None.


4. **T**
   - Opis: Liczba bazowych uczących się, równa także liczbie iteracji.
   - Możliwe wartości: Dowolna liczba całkowita większa od 0.


5. **sample_percent**
   - Opis: Procent próbek pobieranych w każdej iteracji jako proporcja pozostałych nieoznakowanych próbek.
   - Możliwe wartości: Liczba zmiennoprzecinkowa od 0 do 1.


6. **sigma_percentile**
   - Opis: Parametr skali używany w kernelu 'rbf'.
   - Możliwe wartości: Dowolna liczba zmiennoprzecinkowa większa od 0.


7. **similarity_kernel**
   - Opis: Typ kernela, 'rbf', 'knn' lub funkcja.
   - Możliwe wartości: 'rbf', 'knn', 'linear' lub funkcja.


8. **gamma**
   - Opis: Wartość gamma dla kernela 'rbf'.
   - Możliwe wartości: Dowolna liczba zmiennoprzecinkowa większa od 0.

In [10]:
from LAMDA_SSL.Algorithm.Classification.SemiBoost import SemiBoost
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
#datasets = ['cifar10','imdb', 'breast_cancer']
datasets = ['imdb', 'breast_cancer']
labeled_size = 0.3
T=10
base_estimators = [KNeighborsClassifier(n_neighbors=7), RandomForestClassifier(), LinearSVC(max_iter=10**4)]
models = {dataset: SemiBoost(T=T,base_estimator=base_estimators[1]) for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing imdb
X_labeled shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_labeled shape: (7500,)
Training model on imdb
X_train shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_train shape: (7500,)
imdb Results: {'accuracy': 0.744, 'precision': 0.9117170626349892, 'recall': 0.54032, 'f1_score': 0.6785211975085392}
Training Time imdb: 4773.320457458496

Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
breast_cancer Results: {'accuracy': 0.9649122807017544, 'precision': 0.958904109589041, 'recall': 0.9859154929577465, 'f1_score': 0.9722222222222222}
Training Time breast_cancer: 2.3200249671936035



# <font color='green'>LapSVM</font>

### Hiperparametry

1. **distance_function**
    - Funkcja odległości używana do budowy grafu. Ten parametr jest ważny, gdy neighbor_mode jest None.
    - Możliwe wartości: Zależne od implementacji, ale często obejmują funkcje takie jak 'knn', 'linear', 'rbf' oraz dowolne funkcje zdefiniowane przez użytkownika.


2. **gamma_d**
    - Opis: Parametr kernela związany z distance_function.
    - Możliwe wartości: Dowolna liczba rzeczywista, zależna od specyficznej funkcji odległości.


3. **neighbor_mode**
    - Opis: Tryb krawędzi po zbudowaniu modelu grafu przez k-najbliższych sąsiadów. Opcje to 'connectivity' (zwraca macierz 0-1) i 'distance' (zwraca macierz odległości).
    - Możliwe wartości: 'connectivity', 'distance'.


4. **t**
    - Opis: Parametr używany w obliczeniach, szczególnie gdy distance_function jest 'knn'.
    - Możliwe wartości: Dowolna liczba rzeczywista.


5. **n_neighbor**
    - Opis: Wartość k w k-najbliższych sąsiadów.
    - Możliwe wartości: Dowolna liczba całkowita.


6. **kernel_function**
    - Opis: Funkcja kernela odpowiadająca SVM.
    - Możliwe wartości: Zazwyczaj 'rbf', 'linear' lub dowolna funkcja zdefiniowana przez użytkownika.


7. **gamma_k**
    - Opis: Parametr gamma odpowiadający kernel_function.
    - Możliwe wartości: Dowolna liczba rzeczywista.


8. **gamma_A**
    - Opis: Waga kary za złożoność funkcji.
    - Możliwe wartości: Dowolna liczba rzeczywista.


9. **gamma_I**
    - Opis: Waga kary za gładkość rozkładu danych.
    - Możliwe wartości: Dowolna liczba rzeczywista.

In [14]:
from LAMDA_SSL.Algorithm.Classification.LapSVM import LapSVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
#datasets = ['cifar10','imdb', 'breast_cancer']
#datasets = ['imdb', 'breast_cancer']
datasets = ['breast_cancer']
labeled_size = 0.3
models = {dataset: LapSVM() for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
[1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.]
breast_cancer Results: {'accuracy': 0.9473684210526315, 'precision': 0.9452054794520548, 'recall': 0.971830985915493, 'f1_score': 0.9583333333333334}
Training Time breast_cancer: 0.2386949062347412



# <font color='green'>Ladder networks</font>

# <font color='green'>$\Pi$-model</font>

# <font color='green'>Temporal ensembling</font>

### Hiperparametry TemporalEnsembling

- `lambda_u` (waga straty nienadzorowanej): 
  - Możliwe wartości: dowolna liczba zmiennoprzecinkowa.

- `ema_weight` (waga aktualizacji dla eksponencjalnie ważonej średniej ruchomej pseudoetykiet):
  - Możliwe wartości: dowolna liczba zmiennoprzecinkowa między 0 a 1.

- `warmup` (koniec okresu rozgrzewki):
  - Możliwe wartości: procent całkowitej liczby iteracji, np. 0.4 oznacza 40% całkowitej liczby iteracji.

- `num_classes` (liczba klas):
  - Możliwe wartości: dowolna liczba całkowita.

- `num_samples` (liczba próbek):
  - Możliwe wartości: dowolna liczba całkowita.

# <font color='green'>Mean teacher</font>

### Hiperparametry

- **lambda_u**
    - Waga straty nienadzorowanej. Możliwe wartości zależą od specyfiki zadania, ale często wybiera się wartości z zakresu 0.1 do 1.

- **warmup**
    - Końcowy punkt okresu rozgrzewki. Jeśli num_it_total wynosi 100, a warmup jest ustawione na 0.4, to rozgrzewka odbywa się w pierwszych 40 iteracjach. Możliwe wartości to liczby z zakresu od 0 do 1.

- **ema_decay**
    - Współczynnik zaniku dla eksponencjalnego średniego ruchomego. Typowe wartości to liczby z zakresu od 0.9 do 0.999.

- **weight_decay**
    - Współczynnik regularyzacji, który pomaga zapobiegać przeuczeniu. Zwykle wybiera się wartości od 0.0001 do 0.01.

# <font color='green'>VAT Virtual Adversarial Training</font>

### Hiperparametry

- **lambda_u** (waga straty nienadzorowanej):
    - Możliwe wartości: liczby zmiennoprzecinkowe, zazwyczaj w zakresie od 0 do 1.


- **eps** (poziom szumu):
    - Możliwe wartości: małe liczby zmiennoprzecinkowe, zazwyczaj poniżej 1.


- **warmup** (koniec okresu rozgrzewki):
    - Możliwe wartości: liczby zmiennoprzecinkowe z zakresu od 0 do 1, wskazujące na proporcję liczby iteracji, w których odbywa się rozgrzewka.


- **xi** (parametr skali dla zakłóceń):
    - Możliwe wartości: małe liczby zmiennoprzecinkowe, często bliskie 0.


- **lambda_entmin** (waga straty z minimalizacji entropii):
    - Możliwe wartości: liczby zmiennoprzecinkowe, zwykle w zakresie od 0 do 1.

# Transductive
## <font color='green'>TSVM</font>

### Hiperparametry

- `Cl` (Waga próbek oznaczonych)
    - Dowolna wartość liczby rzeczywistej.


- `Cu` (Waga próbek nieoznaczonych)
    - Dowolna wartość liczby rzeczywistej.


- `kernel` (Typ jądra)
    - Możliwe wartości to 'rbf', 'knn' lub funkcja wywoływalna.


- `degree` (Stopień wielomianu)
    - Dotyczy jądra 'poly', wartości całkowite.


- `gamma` (Parametr gamma)
    - Dotyczy jąder 'rbf', 'poly', 'sigmoid', dowolna wartość liczby rzeczywistej.


- `coef0` (Stała składowa funkcji jądra)
    - Dotyczy jąder 'poly' i 'sigmoid', dowolna wartość liczby rzeczywistej.


- `shrinking` (Użycie heurystyki skurczania)
    - Wartości `True` lub `False`.


- `probability` (Wagi dla klasyfikacji kąta obrotu)
    - Dowolna wartość liczby rzeczywistej.


- `tol` (Tolerancja na zatrzymanie treningu)
    - Dowolna wartość liczby rzeczywistej, domyślnie 1e-3.


- `cache_size` (Rozmiar pamięci podręcznej funkcji jądra)
    - Dowolna wartość liczby rzeczywistej.


- `class_weight` (Wagi różnych klas)
    - Słownik lub 'balanced'.


- `max_iter` (Maksymalna liczba iteracji)
    - Wartość całkowita, `-1` dla nieograniczonej liczby.


- `decision_function_shape` (Kształt funkcji decyzyjnej)
    - 'ovo' lub 'ovr', domyślnie 'ovr'.


- `break_ties` (Decyzja w przypadku remisu)
    - Wartości `True` lub `False`.

In [17]:
from LAMDA_SSL.Algorithm.Classification.TSVM import TSVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
#datasets = ['cifar10','imdb', 'breast_cancer']
#datasets = ['imdb', 'breast_cancer']
datasets = ['breast_cancer']
labeled_size = 0.3
models = {dataset: TSVM() for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
[1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1
 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1
 0 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 0 1
 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 0 1 1 1 0
 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1
 1 1 1 0 0 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 1 1 0
 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0]


ValueError: Found input variables with inconsistent numbers of samples: [114, 319]

# <font color='green'>Label propagation algorithm for inference on graphs</font>

### Hiperparametry

- **kernel**
    - Opis: Funkcja jądra, która może być podana jako ciąg znaków 'rbf' lub 'knn', lub jako funkcja.
    - Możliwe wartości: 'rbf', 'knn', funkcja.


- **gamma**
    - Opis: Wartość gamma, gdy funkcja jądra to jądro RBF.
    - Możliwe wartości: Liczba zmiennoprzecinkowa (float).


- **n_neighbors**
    - Opis: Liczba sąsiadów, gdy funkcja jądra to jądro 'n_neighbors'.
    - Możliwe wartości: Liczba całkowita (int).


- **max_iter**
    - Opis: Maksymalna liczba iteracji.
    - Możliwe wartości: Liczba całkowita (int).


- **tol**
    - Opis: Tolerancja zbieżności.
    - Możliwe wartości: Liczba zmiennoprzecinkowa (float).


- **n_jobs**
    - Opis: Liczba równoległych zadań.
    - Możliwe wartości: Liczba całkowita (int) lub None.